# Combined Titanic Submission

## Import / Read In CSV Files

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

import pandas as pd
import matplotlib.pyplot as plt

train = pd.read_csv("datasets/titanicdata/train.csv")
test = pd.read_csv("datasets/titanicdata/test.csv")

## Helper Function to Split Data

In [2]:
def process_age(df,cut_points,label_names):
    df["Age"] = df["Age"].fillna(-0.5)
    df["Age_categories"] = pd.cut(df["Age"],cut_points,labels=label_names)
    return df

def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

In [3]:
cut_points = [-1,0, 5, 12, 18, 35, 60, 100]
label_names = ["Missing", 'Infant', "Child", 'Teenager', "Young Adult", 'Adult', 'Senior']

train = process_age(train,cut_points,label_names)
test = process_age(test,cut_points,label_names)

train = create_dummies(train,"Pclass")
test = create_dummies(test,"Pclass")
for dataset in train:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
for dataset in test:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
#train = create_dummies(train,"Sex")
#test = create_dummies(test,"Sex")
train = create_dummies(train,"Age_categories")
test = create_dummies(test,"Age_categories")

train.head(5)

TypeError: string indices must be integers

## Show Age Categories

### Age Groups

In [ ]:
age_cat_pivot = train.pivot_table(index="Age_categories",values="Survived")
age_cat_pivot.plot.bar()
plt.show()

### Histogram of Deaths by Age

In [ ]:
survived = train[train["Survived"] == 1]
died = train[train["Survived"] == 0]
survived["Age"].plot.hist(alpha=0.5,color='red',bins=50)
died["Age"].plot.hist(alpha=0.5,color='blue',bins=50)
plt.legend(['Survived','Died'])
plt.show()